In [1]:
#找出各母體詞頻
import codecs
import os

SC_num = 0
FC_num = 0
lei_diary = 0
lei_social = 0
lei_article = 0

SC_path = 'D:\\課業相關\\論文資料\\SCS2\\'
FC_path = 'D:\\課業相關\\論文資料\\雷震處理資料\\source\\自由中國(2nd)\\自由中國-非文藝類\\'
lei_path = 'D:\課業相關\\論文資料\\論文程式\\condicate\\topic\\'

'''for file in os.listdir(SC_path):
    with codecs.open(SC_path+file,'rb','utf8') as f:
        head = f.readline()
        content = f.readline().strip().split()
        SC_num += len(content)
        
for file in os.listdir(FC_path):
    with codecs.open(FC_path+file,'rb','utf8') as f:
        head = f.readline()
        content = f.readline().strip().split()
        FC_num += len(content)
        
for file in os.listdir(lei_path):
    with codecs.open(lei_path+file,'rb','utf8') as f:
        head = f.readline()
        content = f.readline().strip().split()
        
        if '日記' in file:
            lei_diary += len(content)        
        elif '文章' in file:
            lei_article += len(content)
        elif '社論' in file:
            lei_social += len(content)
            
print ('總詞數')
print ('平衡語料庫：',SC_num)
print ('自由中國：',FC_num)
print ('雷震日記：',lei_diary)
print ('雷震社論：',lei_social)
print ('雷震文章：',lei_article)'''

"for file in os.listdir(SC_path):\n    with codecs.open(SC_path+file,'rb','utf8') as f:\n        head = f.readline()\n        content = f.readline().strip().split()\n        SC_num += len(content)\n        \nfor file in os.listdir(FC_path):\n    with codecs.open(FC_path+file,'rb','utf8') as f:\n        head = f.readline()\n        content = f.readline().strip().split()\n        FC_num += len(content)\n        \nfor file in os.listdir(lei_path):\n    with codecs.open(lei_path+file,'rb','utf8') as f:\n        head = f.readline()\n        content = f.readline().strip().split()\n        \n        if '日記' in file:\n            lei_diary += len(content)        \n        elif '文章' in file:\n            lei_article += len(content)\n        elif '社論' in file:\n            lei_social += len(content)\n            \nprint ('總詞數')\nprint ('平衡語料庫：',SC_num)\nprint ('自由中國：',FC_num)\nprint ('雷震日記：',lei_diary)\nprint ('雷震社論：',lei_social)\nprint ('雷震文章：',lei_article)"

In [2]:
#找出各母體語言特徵向量相對頻率
import time

desktop_path = 'C:\\Users\\user\\Desktop\\'

feature_path = 'D:\\課業相關\\論文資料\\論文程式\\language_feature\\最終版\\'

vector_base = ['SC','FC','lei']
feature_condicate = ['高頻','bigram','trigram','標點','N+N','N+V','VH+N','D+V','否定','程度','情態']

all_feature_num = []
all_feature = {}
for i in vector_base:
    temp_path = feature_path
    base_sum = 0
    if i == 'SC':
        temp_path += '平衡語料庫\\'
        base_sum = SC_num
    elif i == 'FC':
        temp_path += '自由中國\\'
        base_sum = FC_num
    elif i == 'lei':
        temp_path += '雷震文本\\'
        base_sum = lei_diary+lei_article+lei_social
        
    file_list = []
    for j in feature_condicate:
        for file in os.listdir(temp_path):
            if j in file:
                file_list.append(file)
                break
    
    '''for index,j in enumerate(file_list):
        feature_num = [] 
        with codecs.open(temp_path+j,'rb','utf8') as f: #抓取基準特徵
            for k in f.readlines():
                if k.strip() != '':
                    feature_num.append(str(round(int(k.strip().split(',')[-1])*1000000/base_sum)))
        if len(feature_num) != 100 and '標點' not in j:
            print (j,'ERROR')
        else:
            feature_num.insert(0,i+' '+feature_condicate[index])
            all_feature_num.append(feature_num)'''
    
    for index,j in enumerate(file_list):
        feature_list = [] 
        with codecs.open(temp_path+j,'rb','utf8') as f: #抓取基準特徵
            for k in f.readlines():
                if k.strip() != '':
                    if len(k.strip().split(',')) == 2:
                        if '\ufeff' in k: #去掉開頭BOM
                            k = k.replace('\ufeff','')
                        feature_list.append(k.strip().split(',')[0])
                    else:
                        feature_list.append('')
        if len(feature_list) != 100 and '標點' not in j:
            print (j,'ERROR')
        else:
            #feature_list.insert(0,i+' '+feature_condicate[index])
            all_feature[i+' '+feature_condicate[index]] = feature_list
            
'''with codecs.open(desktop_path+'feature_base_vector.csv','wb','utf8') as g:
    for i in all_feature_num:
        g.write(','.join(i)+'\r\n')'''

'''with codecs.open(desktop_path+'feature_base_list.csv','wb','utf8') as g:
    for i in vector_base:
        for j in feature_condicate:
            g.write(i+' '+j+','+','.join(all_feature[i+' '+j])+'\r\n')'''
        
print ('END')

END


In [3]:
#比較各類別分布表現
import codecs
import os
from collections import defaultdict
import time
import numpy as np
from sklearn.preprocessing import MinMaxScaler

desktop_path = 'C:\\Users\\user\\Desktop\\'

feature_path = 'D:\\課業相關\\論文資料\\論文程式\\language_feature\\最終版\\'

vector_base = ['SC','FC','lei']
feature_condicate = ['高頻','bigram','trigram','標點','N+N','N+V','VH+N','D+V','否定','程度','情態']

test_vector = 'C:\\Users\\user\\Desktop\\RF result\\vector\\'

all_vector_space = {}

base_value = defaultdict(list)
with codecs.open(desktop_path+'feature_base_vector.csv','rb','utf8') as f:
    for line in f.readlines(): 
        if line.strip() == '':
            continue
        base_value[line.strip().split(',')[0]] = list(map(int,map(float,line.strip().split(',')[1:])))

#各類別文本平均值與母體平均之差異
for vb in vector_base:
    for fc in feature_condicate:
        for cs in ['name','topic']:
            
            if (vb == 'FC' and cs == 'topic') or (vb == 'lei' and cs == 'name'):
                continue
      
            classification_num = defaultdict(dict) #類別-詞組-文本數值
        
            feature_num = 0

            with codecs.open(test_vector+vb+' '+fc+' '+cs+'.csv','rb','utf8') as f: #將各類別的詞組整合成同一陣列
                for i in f.readlines():
                    if i.strip() == '':
                        continue
                    line = i.strip().split(',')
                    feature_num = len(line) - 1
                    for index,j in enumerate(line[1:]):
                        if str(index) not in classification_num[line[0]]:
                            classification_num[line[0]][str(index)] = [j]
                        else:
                            classification_num[line[0]][str(index)].append(j)
                            
            #vector_space = np.zeros((len(classification_num)+1,feature_num),np.float64) #最後一列為母體基準
            vector_space = np.zeros((len(classification_num),feature_num),np.float64) 

            for k,v in classification_num.items():
                #temp_rsd = []
                temp_avg = []
                for i in range(len(v)):
                    temp_num = list(map(int,map(float,v[str(i)])))

                    feature_base_value = base_value[vb+' '+fc][i]

                    
                    '''rsd = [(x-feature_base_value)**2 for x in temp_num] 
                    rsd = (sum(rsd)/len(rsd))**0.5/feature_base_value #這邊採用N而非N-1作為標準差分母
                    #if rsd <= 1:
                    #    rsd = 0
                    temp_rsd.append(str(round(rsd,5)))'''
                    
                    avg = sum(temp_num)/len(temp_num) #計算平均，不考慮0
                    temp_avg.append(str(round(avg-feature_base_value,5)))
                    
                    vector_space[int(k),i] = avg-feature_base_value
                    
            print (vector_space[:,(1,2)])
                    
            min_max_scaler = MinMaxScaler(feature_range = (0,1))
            vector_space = min_max_scaler.fit_transform(vector_space)
            
            print (vector_space[:,(1,2)])
            
            '''for i in range(vector_space.shape[0]):
                for j in range(vector_space.shape[1]):
                    vector_space[i,j] = round(vector_space[i,j] - vector_space[vector_space.shape[0]-1,j],5)'''
                    
            #print (vector_space[:,(1,2)])
            #print (list(map(str,list(vector_space[0]))))
            
            all_vector_space[vb+' '+fc+' '+cs] = vector_space
                
            '''with codecs.open(desktop_path+'avg_feature.csv','ab','utf8') as a:
                for i in range(len(classification_num)):
                    a.write(vb+' '+fc+' '+cs+' '+str(i)+','+','.join(list(map(str,list(vector_space[i]))))+'\r\n')'''
            time.sleep(1)
print ('END')

[[  918.29213483 -1714.34831461]
 [ 3315.025        586.6875    ]
 [ 4724.9047619   -304.11111111]
 [ -600.8125      1252.97916667]
 [-4554.70731707  2056.14634146]
 [-4938.63333333 -1333.93333333]
 [ 5426.31034483   639.34482759]
 [ 2861.7037037   1239.33333333]
 [ 6058.73076923 -4667.76923077]]
[[ 0.53257539  0.43924123]
 [ 0.75051242  0.78145787]
 [ 0.87871403  0.64897575]
 [ 0.39444187  0.88055068]
 [ 0.03491073  1.        ]
 [ 0.          0.49581763]
 [ 0.94249345  0.78928922]
 [ 0.70929151  0.87852123]
 [ 1.          0.        ]]


KeyboardInterrupt: 

In [15]:
#目的找出 1. 類別顯著於母體 2. 類別彼此接近但又非一般
classification_name = ['雷震','殷海光','夏道平','傅正','龍平甫','蔣勻田','朱伴耘','胡適','羅鴻詔']
classification_topic = ['社論','文章','日記']

out_feature = ['高頻','bigram','trigram','標點']

for k,v in all_vector_space.items(): #逐語言特徵
    
    if k.split()[1] in out_feature or k.split()[2] == 'name':
        continue
    
    aspect = []
    key = k.split()[0]+' '+k.split()[1]
    #print (key)
    print (k)
    
    if k.split()[2] == 'name':
        aspect = classification_name
    elif k.split()[2] == 'topic':
        aspect = classification_topic
        
    similar = defaultdict(list)
    non_similar = defaultdict(list)
    feature_class_group = defaultdict(list) #各詞彙分群組合
        
    feature_select = all_feature[key]
    for i in range(v.shape[1]): #逐詞組
        temp_list = v[:,i][:-1]
        temp_group = []
        
        if max(temp_list) == min(temp_list) or max(temp_list) - min(temp_list) < 0.3: #不考慮皆為0及彼此差異不大的類別
            continue
        for a in range(len(temp_list)): #交叉運算
            for b in range(len(temp_list)-a-1):
                t = b + a + 1
                
                if abs(temp_list[a] - temp_list[t]) < 0.33:# and abs(temp_list[a] - temp_list[t]) != 0:
                    #if abs(temp_list[a]) < 0.33 or abs(temp_list[t]) < 0.33:
                    #    continue
                    #print (feature_select[i],aspect[a],a,aspect[t],t,'相似',abs(temp_list[a] - temp_list[t]))
                    similar[(aspect[a],aspect[t])].append(feature_select[i])                   
                elif abs(temp_list[a] - temp_list[t]) > 0.66:
                    #print (feature_select[i],aspect[a],a,aspect[t],t,'不相似',abs(temp_list[a] - temp_list[t]))
                    non_similar[(aspect[a],aspect[t])].append(feature_select[i])
        
        #分群
        for a in range(len(temp_list)):
            all_check = True
            wait_append = []
            for b in temp_group:
                if type(b) == int and abs(temp_list[a] - temp_list[b]) < 0.33:
                    temp_group.remove(b)
                    wait_append.append([b,a])
                    all_check = False
                elif type(b) == list:
                    check = True
                    for c in b:
                        if abs(temp_list[a] - temp_list[c]) >= 0.33:
                            check = False
                            break
                    if check:
                        b.append(a)
                        all_check = False
            for d in wait_append:
                temp_group.append(d)
            if all_check:
                temp_group.append(a)
        
        #找出獨特群
        for e in temp_group:
            if type(e) == int:
                check = True
                for a in range(len(temp_list)):
                    if e != a:
                        if abs(temp_list[a] - temp_list[e]) < 0.66:
                            check = False
                            break
                if check:
                    temp_group.remove(e)
                    print (temp_list[e])
                    temp_group.append([e])
                
        print (feature_select[i],temp_group)
        
        time.sleep(0.5)
    '''so = sorted(similar.items(), key=lambda d:len(d[1]), reverse = True)
    with codecs.open(desktop_path+'相似.csv','ab','utf8') as a:
        
        print ('相似')
        index = 0
        for x,y in so:
            index += 1
            if index == 20:
                break
            print (x,y)
        print ()
        
        a.write(key+'\r\n')
        for x,y in so:
            a.write(x[0]+' '+x[1]+','+','.join(y)+'\r\n')
        a.write('\r\n')
         
    so = sorted(non_similar.items(), key=lambda d:len(d[1]), reverse = True)
    with codecs.open(desktop_path+'不相似.csv','ab','utf8') as a:
        
        print ('不相似')
        index = 0
        for x,y in so:
            index += 1
            if index == 20:
                break
            print (x,y)
        print ()
        
        a.write(key+'\r\n')
        for x,y in so:
            a.write(x[0]+' '+x[1]+','+','.join(y)+'\r\n')
        a.write('\r\n')'''
        
print ('END')

SC VH+N topic
0.75769
先進(VH)國家(Na) [[1, 2], [0]]
國營(VH)事業(Na) [0, 1, 2]
0.55378
大(VH)企業(Na) [[1, 2], [0]]
大(VH)問題(Na) [1, [0, 2]]
小(VH)事(Na) [1, [0, 2]]
新(VH)世紀(Na) [1, [0, 2]]
公營(VH)事業(Na) [0, 1, 2]
0.69543
大(VH)飯店(Nc) [[0, 1], [2]]
0.0411
小(VH)狗(Na) [[0, 1], [2]]
大(VH)醫院(Nc) [[0, 1], 2]
重要(VH)因素(Na) [0, 1, 2]
0.77358
受難(VH)者(Na) [[1, 2], [0]]
-0.21311
密切(VH)關係(Na) [[0, 2], [1]]
-0.38217
自由(VH)主義(Na) [[1, 2], [0]]
小(VH)動物(Na) [1, [0, 2]]
大(VH)公司(Nc) [1, [0, 2]]
小(VH)船(Na) [[0, 1], 2]
0.23174
大(VH)城市(Na) [[0, 2], [1]]
不同(VH)意見(Na) [0, [1, 2]]
大(VH)家庭(Na) [[0, 1], 2]
0.58503
大(VH)都市(Na) [[0, 2], [1]]
好(VH)方法(Na) [1, [0, 2]]
0.35345
多(VH)人(Na) [[0, 1], [2]]
-0.23333
好(VH)消息(Na) [[0, 1], [2]]
0.83529
好(VH)話(Na) [[1, 2], [0]]
0.18919
退伍(VH)軍人(Na) [[0, 1], [2]]
lei D+V topic
0.22019
不(D)好(VH) [[0, 1], [2]]
0.37868
又(D)說(VE) [[0, 1], [2]]
0.29218
不(D)知(VK) [[0, 1], [2]]
0.79503
不(D)敢(VL) [[1, 2], [0]]
0.17658
甚(Dfa)久(VH) [[0, 1], [2]]
0.28306
很(Dfa)好(VH) [[0, 1], [2]]
0.17027
不(D)贊成(VK) [[0,

KeyboardInterrupt: 